# TestPyPI Install Verification
Tests that `quantem-widget` installed from TestPyPI works correctly.
Run this in the `test-widget-env` conda environment.

In [ ]:
try:
    %load_ext autoreload
    %autoreload 2
    %env ANYWIDGET_HMR=1
except Exception:
    pass  # autoreload unavailable (Colab Python 3.12+)

In [ ]:
import quantem.widget
print(f"quantem-widget location: {quantem.widget.__file__}")
print(f"Available widgets: {[x for x in dir(quantem.widget) if not x.startswith('_')]}")
print(f"quantem.widget {quantem.widget.__version__}")

In [ ]:
import numpy as np
# --- 2D HRTEM image ---
def make_hrtem(size=256, seed=0):
    rng = np.random.default_rng(seed)
    y, x = np.mgrid[:size, :size]
    img = np.zeros((size, size))
    freqs = [(0.08, 0.0), (0.06, np.pi / 3), (0.10, np.pi / 6)]
    for freq, angle in freqs:
        img += np.cos(2 * np.pi * freq * (x * np.cos(angle) + y * np.sin(angle)))
    r = np.sqrt((x - size // 2)**2 + (y - size // 2)**2)
    envelope = 1.0 / (1 + np.exp(-0.05 * (size // 3 - r)))
    img = img * envelope + rng.normal(0, 0.3, (size, size))
    return img.astype(np.float32)
imgs_2d = [make_hrtem(256, seed=i) for i in range(6)]
# --- 3D focal series ---
def make_focal_series(n_frames=30, size=256):
    y, x = np.mgrid[:size, :size]
    particles = [
        (size * 0.35, size * 0.4, 18, 1.0),
        (size * 0.65, size * 0.55, 25, 0.7),
        (size * 0.45, size * 0.7, 12, 1.2),
        (size * 0.7, size * 0.3, 15, 0.9),
    ]
    defocus = np.linspace(-60, 60, n_frames)
    frames = np.zeros((n_frames, size, size), dtype=np.float32)
    for f_idx, df in enumerate(defocus):
        frame = np.full((size, size), 0.5)
        for cx, cy, r, z in particles:
            dist = np.sqrt((x - cx) ** 2 + (y - cy) ** 2)
            edge = 1.0 / (1 + np.exp((dist - r) * 2))
            if abs(df) > 3:
                fresnel = np.cos(0.005 * df * (dist - r) ** 2) * np.exp(
                    -((dist - r) ** 2) / (2 * (3 + abs(df) * 0.15) ** 2)
                )
                frame += z * (edge * 0.3 + fresnel * 0.2 * np.sign(df))
            else:
                frame += z * edge * 0.4
        frame += np.random.normal(0, 0.03, (size, size))
        frames[f_idx] = frame
    return frames
focal_stack = make_focal_series()
# --- 3D volume (core-shell nanoparticle) ---
def make_volume(size=64):
    z, y, x = np.mgrid[:size, :size, :size]
    c = size // 2
    r = np.sqrt((x - c)**2 + (y - c)**2 + (z - c)**2)
    vol = np.zeros((size, size, size), dtype=np.float32)
    vol[r < size * 0.4] = 0.5   # shell
    vol[r < size * 0.25] = 1.0  # core
    vol += np.random.normal(0, 0.05, vol.shape).astype(np.float32)
    return vol
volume = make_volume()
# --- 4D-STEM data ---
def make_4dstem(scan=32, det=64):
    data = np.random.poisson(2, (scan, scan, det, det)).astype(np.float32)
    cy, cx = det // 2, det // 2
    y, x = np.mgrid[:det, :det]
    r = np.sqrt((x - cx)**2 + (y - cy)**2)
    disk = np.exp(-r**2 / (2 * 8**2)) * 50
    data += disk[None, None, :, :]
    # Add some Bragg spots
    for dy, dx in [(10, 0), (-10, 0), (0, 10), (0, -10)]:
        spot = np.exp(-((x - cx - dx)**2 + (y - cy - dy)**2) / (2 * 2**2)) * 30
        data[:, :, :, :] += spot[None, None, :, :]
    return data
stem_data = make_4dstem()
print(f"2D images: {len(imgs_2d)} x {imgs_2d[0].shape}")
print(f"Focal stack: {focal_stack.shape}")
print(f"Volume: {volume.shape}")
print(f"4D-STEM: {stem_data.shape}")

## Show2D

In [ ]:
from quantem.widget import Show2D
Show2D(imgs_2d[0], title="Single HRTEM Image", cmap="gray")

In [ ]:
Show2D(imgs_2d, title="Gallery — 3 Columns", ncols=3, cmap="viridis")

## Show3D

In [ ]:
from quantem.widget import Show3D
defocus_values = np.linspace(-60, 60, 30)
labels = [f"C10={df:.0f} nm" for df in defocus_values]
Show3D(
    focal_stack,
    labels=labels,
    title="Through-Focus Series",
    cmap="gray",
    pixel_size=0.25,
    fps=8,
)

## Show3DVolume

In [ ]:
from quantem.widget import Show3DVolume
Show3DVolume(volume, title="Core-Shell Nanoparticle", cmap="inferno")

## Show4DSTEM

In [ ]:
from quantem.widget import Show4DSTEM
Show4DSTEM(stem_data, title="4D-STEM with Bragg Spots", cmap="viridis")

##Mark2D

In [ ]:
fromquantem.widgetimportMark2Ddefmake_haadf_stem(size=256,spacing=18,sigma=2.8):y,x=np.mgrid[:size,:size]img=np.random.normal(0.08,0.015,(size,size))a1=np.array([spacing,0.0])a2=np.array([spacing*0.5,spacing*np.sqrt(3)/2])foriinrange(-1,size//spacing+2):forjinrange(-1,size//spacing+2):cx=i*a1[0]+j*a2[0]cy=i*a1[1]+j*a2[1]if-spacing<cx<size+spacingand-spacing<cy<size+spacing:intensity=0.7+0.3*((i+j)%3==0)img+=intensity*np.exp(-((x-cx)**2+(y-cy)**2)/(2*sigma**2))scan_noise=np.random.normal(0,0.01,(size,1))*np.ones((1,size))img+=scan_noisereturnnp.clip(img,0,None).astype(np.float32)haadf=make_haadf_stem()w_mark2d=Mark2D(haadf,scale=1.0,max_points=5)w_mark2d

### Retrieve selected points
After clicking on atom columns above, run the cell below to get pixel coordinates.

In [ ]:
points=w_mark2d.selected_pointsprint(f"Selected{len(points)}point(s):")fori,pinenumerate(points):print(f"P{i}:row={p['row']:.1f},col={p['col']:.1f}")iflen(points)>=3:origin=np.array([points[0]["row"],points[0]["col"]])u=np.array([points[1]["row"],points[1]["col"]])-originv=np.array([points[2]["row"],points[2]["col"]])-originprint(f"\nLatticebasis(fromfirst3points):")print(f"origin=(row={origin[0]:.1f},col={origin[1]:.1f})")print(f"u=({u[0]:.1f},{u[1]:.1f})|u|={np.linalg.norm(u):.1f}px")print(f"v=({v[0]:.1f},{v[1]:.1f})|v|={np.linalg.norm(v):.1f}px")

## All Done!
If all widgets above rendered correctly, the TestPyPI install is working.